### FE621 - Homework #1

**Author**: Sid Bhatia

**Date**: February 11th, 2023

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Dr. Sveinn Olafsson

**TA**: Mr. Dong Woo Kim

#### Problem #1 - Analyzing Options Data

1. Collect Data:

- Download market prices and implied volatilities for S&P 500 index options (SPX options). You also need the value of the S&P 500 index. The data can be obtained from, e.g., Yahoo Finance or Bloomberg.
- Download risk-free interest rate data from http://www.federalreserve.gov/releases/H15/Current/.

In [46]:
from yahoo_fin import options

# Use "^SPX" for the S&P 500 Index.
ticker_symbol = '^SPX'

# Try-except to retrieve options data for ^SPX.
try:
    # Get all available expiration dates for the ticker.
    expiration_dates = options.get_expiration_dates(ticker_symbol)
    
    if expiration_dates:
        # Get options chain for the nearest expiration date.
        options_chain = options.get_options_chain(ticker_symbol, expiration_dates[0])

        calls = options_chain['calls']
        puts = options_chain['puts']

        print("Calls Data:\n", calls.head())  # Display the first few rows of call options data.
        print("\nPuts Data:\n", puts.head())  # Display the first few rows of put options data.
    else:
        print("No expiration dates found for ticker:", ticker_symbol)
except Exception as e:
    print("Error retrieving options data:", str(e))

Calls Data:
          Contract Name         Last Trade Date  Strike  Last Price      Bid   
0  SPXW240212C01200000   2023-12-15 9:47AM EST  1200.0     3516.61  3575.50  \
1  SPXW240212C02000000   2024-02-11 8:32PM EST  2000.0     3027.00     0.00   
2  SPXW240212C02400000   2024-02-08 8:20PM EST  2400.0     2597.00     0.00   
3  SPXW240212C02600000   2024-02-09 4:02PM EST  2600.0     2425.60     0.00   
4  SPXW240212C04000000  2024-02-09 12:52PM EST  4000.0     1017.30  1022.20   

      Ask   Change % Change Volume Open Interest Implied Volatility  
0  3585.4     0.00        -      2             0              0.00%  
1     0.0    32.40   +1.08%      2             0              0.00%  
2     0.0  2597.00        -      -             -              0.00%  
3     0.0  2425.60        -      -             -              0.00%  
4  1029.8   147.78  +17.00%      8             0            208.20%  

Puts Data:
          Contract Name         Last Trade Date  Strike  Last Price   Bid   
0  

In [62]:
import pandas as pd

# Load the CSV file for the risk-free rate.
rfr_df = pd.read_csv('FRB_H15.csv')

# Display the first few rows of the dataframe.
print(rfr_df.head())
print(rfr_df.tail())

print(rfr_df.columns)

# Drop NAs in risk-free rate data frame.
rfr_df_nonan = rfr_df.dropna(subset=['Market yield on U.S. Treasury securities at 3-month  constant maturity, quoted on investment basis'])

# Retrieve latest risk-free rate based on 3-month Treasury as of 2/8.
rfr = rfr_df_nonan['Market yield on U.S. Treasury securities at 3-month  constant maturity, quoted on investment basis'].iloc[-1]

print("Current Risk-Free Interest Rate as of 2/8:", rfr)

   Series Description   
0               Unit:  \
1         Multiplier:   
2           Currency:   
3  Unique Identifier:   
4         Time Period   

  Market yield on U.S. Treasury securities at 1-month  constant maturity, quoted on investment basis   
0                                  Percent:_Per_Year                                                  \
1                                                  1                                                   
2                                                NaN                                                   
3                             H15/H15/RIFLGFCM01_N.B                                                   
4                                     RIFLGFCM01_N.B                                                   

  Market yield on U.S. Treasury securities at 3-month  constant maturity, quoted on investment basis   
0                                  Percent:_Per_Year                                                  \
1               

2. Write a function that computes implied volatilities:

- Implement a function that computes the Black-Scholes prices of call and put options with parameters $S_0$ (stock price), $\sigma$ (vol), $\tau = T- t$ (time to maturity), $K$ (strike), $r$ (interest rate), and $\delta$ (dividend yield).
- Implement a function that uses Newton’s method to compute the implied volatility of call and put options.
Provide pseudocode for your approach (i.e., provide step-by-step algorithmic instructions).
- *Note: Newton’s method requires computing the derivative of the Black-Scholes price with respect to the
volatility σ. This derivative is known as vega and it has a closed-form formula in the Black-Scholes model.*

**Black-Scholes Price Calculation Pseudocode**

```plaintext
function black_scholes(S_0, K, r, tau, sigma, delta, option_type):
    Calculate d1 and d2 using their formulas
    if option_type is "call":
        C = S_0 * exp(-delta * tau) * N(d1) - K * exp(-r * tau) * N(d2)
        return C
    else if option_type is "put":
        P = K * exp(-r * tau) * N(-d2) - S_0 * exp(-delta * tau) * N(-d1)
        return P

In [65]:
from typing import Union
import numpy as np
import scipy.stats as si

def black_scholes(S_0: float, K: float, r: float, tau: float, sigma: float, delta: float, option_type: str) -> float:
    """
    Calculate the Black-Scholes option price for a call or put option.

    Parameters:
    - S_0 (float): Initial stock price.
    - K (float): Strike price.
    - r (float): Risk-free interest rate.
    - tau (float): Time to maturity (in years).
    - sigma (float): Volatility of the underlying asset.
    - delta (float): Continuous dividend yield.
    - option_type (str): Type of the option ('call' or 'put').

    Returns:
    - float: The Black-Scholes price of the option.
    """
    d1 = (np.log(S_0 / K) + (r - delta + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    
    if option_type == "call":
        price = S_0 * np.exp(-delta * tau) * si.norm.cdf(d1) - K * np.exp(-r * tau) * si.norm.cdf(d2)
    elif option_type == "put":
        price = K * np.exp(-r * tau) * si.norm.cdf(-d2) - S_0 * np.exp(-delta * tau) * si.norm.cdf(-d1)
    else:
        raise ValueError("Option type must be 'call' or 'put'.")
    
    return price

**Implied Volatility Calculation Using Newton's Method**

```plaintext
function compute_implied_volatility(market_price, S_0, K, r, tau, delta, option_type):
    Initialize sigma with an initial guess, e.g., 0.2
    tolerance = 1e-6
    max_iterations = 100
    for i in 1 to max_iterations:
        Calculate the Black-Scholes price for the current sigma
        Compute Vega for the current sigma
        Calculate f(sigma) as the difference between Black-Scholes price and market_price
        Update sigma using Newton's method formula
        if absolute difference in sigma is less than tolerance:
            break
    return sigma

In [67]:
def vega(S_0: float, K: float, r: float, tau: float, sigma: float, delta: float) -> float:
    """
    Calculate Vega of an option, which is the sensitivity of the option price to a change in volatility.

    Parameters:
    - S_0, K, r, tau, sigma, delta: As described in the black_scholes function.

    Returns:
    - float: Vega of the option.
    """
    d1 = (np.log(S_0 / K) + (r - delta + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    return S_0 * np.exp(-delta * tau) * np.sqrt(tau) * si.norm.pdf(d1)

In [68]:
def compute_implied_volatility(market_price: float, S_0: float, K: float, r: float, tau: float, delta: float, option_type: str) -> float:
    """
    Compute the implied volatility of a call or put option using Newton's method.

    Parameters:
    - market_price (float): Market price of the option.
    - S_0, K, r, tau, delta: As described in the black_scholes function.
    - option_type (str): Type of the option ('call' or 'put').

    Returns:
    - float: Implied volatility of the option.
    """
    sigma = 0.2  # Initial guess for volatility.
    tolerance = 1e-6  # Convergence criterion.
    max_iterations = 100  # Maximum number of iterations.
    
    for _ in range(max_iterations):
        # Calculate the Black-Scholes price with the current guess for volatility.
        price = black_scholes(S_0, K, r, tau, sigma, delta, option_type)
        
        # Calculate Vega, the derivative of price with respect to volatility.
        option_vega = vega(S_0, K, r, tau, sigma, delta)
        
        # Difference between market price and model price.
        price_diff = market_price - price
        
        # Prevent division by zero.
        if abs(option_vega) < 1e-8:
            break
        
        # Update guess for sigma using Newton's method.
        sigma_new = sigma + price_diff / option_vega
        
        # Check if the change in sigma is within the tolerance level.
        if abs(sigma_new - sigma) < tolerance:
            return sigma_new
        
        sigma = sigma_new
    
    return sigma

3. Generate implied volatility smiles:

- Use your function in part (b) to compute the implied volatilities of options with the following maturities: 1 month, 3 months, 6 months, 1 year. You may also consider other maturities. For the market price of an option, use the average of the bid and ask prices.
- For each maturity, plot your computed implied volatilities and the downloaded market implied volatilities. Do this for both call and put options. How do the computed and market volatilities compare?
- *Note: Rather than plotting implied volatilities as a function of strike price, you may explore plotting them as a function of the so-called option moneyness. The moneyness is commonly defined as $\frac{S_0}{K}$, or $\frac{ln(\frac{K}{F})}{\sigma_{\text{ATM}}\sqrt{\tau}}$, where $F = S_0e^{r\tau}$ is the forward price, and $\sigma_{\text{ATM}}$ is the implied volatility of the ATM option (i.e., the option with strike $K \approx F$).*

#### Problem #2 - Root Finding Algorithms

1. Write pseudocode for using the bisection method and Newton’s method to iteratively compute the square root
of a positive number $a$. Remember to state how you choose the initial search interval for the bisection method,
and the initial guess for Newton’s method.
- *Hint: Finding the square root of $a$ is equivalent to finding the positive root of the function $f(x) = x^2 - a$.*

**Bisection Method Pseudocode**

The bisection method requires selecting an initial interval $[x_0, x_1]$ that contains the root.

1. *Initial Interval Selection*:
   - Choose `x_0 = 0` and `x_1 = a` if `a >= 1` or `x_1 = 1` if `a < 1`. This ensures the square root of `a` is within the interval `[x_0, x_1]`.

2. *Check Midpoint*:
   - Compute the midpoint `m = (x_0 + x_1) / 2` and evaluate `f(m) = m^2 - a`.

3. *Update Interval*:
   - If `f(m) = 0`, then `m` is the square root, and the process stops.
   - If `f(m) * f(x_0) < 0`, the root is in the interval `[x_0, m]`. Set `x_1 = m`.
   - Otherwise, the root is in the interval `[m, x_1]`. Set `x_0 = m`.

4. *Convergence Check*:
   - Repeat steps 2 and 3 until the interval `[x_0, x_1]` is sufficiently small, e.g., `|x_1 - x_0| < epsilon`, where `epsilon` is a small number representing the desired accuracy.

5. *Result*:
   - The square root of `a` is approximately `(x_0 + x_1) / 2` after convergence.

**Newton's Method Pseudocode**

1. *Initial Guess*:
   - Choose `x_0 = a / 2` if `a >= 1` or `x_0 = 1` if `a < 1`. This is a reasonable starting point for most positive numbers `a`.

2. *Iteration*:
   - Apply the Newton iteration formula to refine the guess: `x_{n+1} = x_n - (f(x_n) / f'(x_n)) = x_n - ((x_n^2 - a) / (2x_n)) = (x_n + (a / x_n)) / 2`.

3. *Convergence Check*:
   - Repeat step 2 until the change in successive estimates is below a threshold, e.g., `|x_{n+1} - x_n| < epsilon`, where `epsilon` is the desired precision.

4. *Result*:
   - The square root of `a` is `x_n` when the process converges.

2. Implement your procedures in part (a) with tolerance $\epsilon = 10^{-6}$.
    1. How many iterations do the algorithms need for $a = 0.5$ and $a = 2$?
    2. To visualize the convergence, plot $|err_n|$ as a function of $n$ ($|err_n|$ should converge to zero), and $\log(|err_n|)$ as a function of $\log(n)$, where $err_n = \sqrt{a} - x_n$ is the error on the $n$-th iteration of the algorithm.
    3. Which algorithm seems to converge faster? Is what you observe in line with theoretical results about the rate of convergence of the two methods?

In [70]:
def sqrt_bisection(a: float, epsilon: float = 1e-6) -> float:
    if a < 0:
        raise ValueError("a must be non-negative")
    elif a == 0 or a == 1:
        return a
    
    lower, upper = (0, a) if a > 1 else (a, 1)
    
    while upper - lower > epsilon:
        midpoint = (lower + upper) / 2
        if midpoint**2 == a:
            return midpoint
        elif midpoint**2 < a:
            lower = midpoint
        else:
            upper = midpoint
    
    return (lower + upper) / 2

In [72]:
def sqrt_newton(a: float, epsilon: float = 1e-6) -> float:
    if a < 0:
        raise ValueError("a must be non-negative")
    elif a == 0 or a == 1:
        return a
    
    x_n = a / 2  # Initial guess
    while True:
        x_next = 0.5 * (x_n + a / x_n)
        if abs(x_next - x_n) < epsilon:
            return x_next
        x_n = x_next

In [74]:
def sqrt_bisection_iterations(a: float, epsilon: float = 1e-6) -> int:
    if a == 0 or a == 1:
        return 0  # No iterations needed
    lower, upper = (0, a) if a > 1 else (a, 1)
    iterations = 0
    while upper - lower > epsilon:
        midpoint = (lower + upper) / 2
        if midpoint**2 < a:
            lower = midpoint
        else:
            upper = midpoint
        iterations += 1
    return iterations

def sqrt_newton_iterations(a: float, epsilon: float = 1e-6) -> int:
    if a == 0 or a == 1:
        return 0  # No iterations needed
    x_n = a / 2
    iterations = 0
    while True:
        x_next = 0.5 * (x_n + a / x_n)
        if abs(x_next - x_n) < epsilon:
            break
        x_n = x_next
        iterations += 1
    return iterations

# Calculate and print the number of iterations for each method and value of a.
for a in [0.5, 2]:
    bisection_iters = sqrt_bisection_iterations(a)
    newton_iters = sqrt_newton_iterations(a)
    print(f"For a = {a}: Bisection method iterations = {bisection_iters}, Newton's method iterations = {newton_iters}")

For a = 0.5: Bisection method iterations = 19, Newton's method iterations = 5
For a = 2: Bisection method iterations = 21, Newton's method iterations = 4
